# Simple RBMK Model
This model is intended to highlight some basic reactor physics properties of an RBMK model.  The simplification will be in the geometric representation of fuel an control elements each of which will be explained in the blocks below.

## Materials

In [1]:
import openmc
import numpy as np


### Fuel Channel Materials

Fuel channels will be modeled as a homogeneous mixture of water, slightly enriched uranium fuel, and ziraloy cladding.  In the block below I will work out the volume fractions of each material.

In [2]:
# computations to estimate the atom density of various 
# nuclides comprising the fuel and water that I am smearing
# together for the fuel channels.

# obviously, this is a "guesstimate" and gross simplification
# but (aspirationally) can be made more accurate for more robust
# calculations

r_channel = 8.0/2.; # cm
L_channel = 700; # cm

V_channel = np.pi*(r_channel**2)*L_channel

n_bundles = 2;
n_pins_per_bundle = 18;
L_bundle = L_channel/n_bundles; # okay; this is a fudge

r_fuel = 1.146/2.; # cm, just the radius of the fuel pellet
r_clad_o = 1.36/2.; # cm, clad outer radius
r_clad_i = r_clad_o - 0.09; # cm, clad inner radius (0.9mm clad thickness)


fuel_vol = np.pi*(r_fuel**2)*L_bundle*n_bundles*n_pins_per_bundle; #cm^3
fuel_vol_frac = fuel_vol/V_channel;
print(f'fuel volume fraction: %5.4f'%fuel_vol_frac);

clad_vol = np.pi*(r_clad_o**2 - r_clad_i**2)*L_bundle*n_bundles*n_pins_per_bundle;
clad_vol_frac = clad_vol/V_channel;
print(f'clad volume fraction: %5.4f'%clad_vol_frac);

coolant_vol_frac = 1 - clad_vol_frac - fuel_vol_frac
print(f'coolant volume fraction: %5.4f'%coolant_vol_frac);


fuel volume fraction: 0.3694
clad volume fraction: 0.1286
coolant volume fraction: 0.5020


with this volume fraction information I can create the fuel channel mixture.

In [3]:
uo2 = openmc.Material(name='uo2');
uo2.add_element('U',1,enrichment=2.2);
uo2.add_element('O',2);
uo2.set_density('g/cm3',10.5);

water = openmc.Material(name='water');
water.add_element('H',2);
water.add_element('O',1);
water.set_density('g/cm3',0.7);

clad = openmc.Material(name='clad');
clad.add_element('Zr',1);
clad.set_density('g/cm3',6.51);

fuel_chan_mat = openmc.Material.mix_materials([uo2,water,clad],
                                             [fuel_vol_frac,
                                             coolant_vol_frac,
                                             clad_vol_frac],'vo');

fuel_chan_mat.add_s_alpha_beta('c_H_in_H2O');


the graphite and B4C control rods will be modeled simply.

In [4]:
b4c = openmc.Material(name='b4c');
b4c.add_element('B',4.);
b4c.add_element('C',1.);
b4c.set_density('g/cm3',2.52);

In [5]:
mod = openmc.Material(name='mod');
mod.add_element('C',1.);
mod.set_density('g/cm3',1.7);
mod.add_s_alpha_beta('c_Graphite');

In [6]:
materials = openmc.Materials()
materials += [fuel_chan_mat,b4c,mod];
materials.export_to_xml();

## Geometry
In the next chunks of code I will create universes for fuel channels and control rod channels.

In [9]:
top = openmc.ZPlane(z0 = L_channel,boundary_type='vacuum');
bottom = openmc.ZPlane(z0=0.,boundary_type='vacuum');

chan_wall = openmc.ZCylinder(r=r_channel);

# fueled universe
fuel_chan_cell = openmc.Cell();
fuel_chan_cell.fill = fuel_chan_mat;
fuel_chan_cell.region = -chan_wall & +bottom & -top;

mod_cell = openmc.Cell();
mod_cell.fill = mod;
mod_cell.region = +chan_wall & +bottom & -top;

fu = openmc.Universe();
fu.add_cells([fuel_chan_cell,mod_cell]);

# control rod universe
pct_insert_cr = 0.5;
cr_follower_len = 50; # cm, length of graphite follower

cr_surf = openmc.ZPlane(z0=pct_insert_cr*L_channel);
crf_surf = openmc.ZPlane(z0=pct_insert_cr*L_channel-cr_follower_len);

cr_cell = openmc.Cell();
cr_cell.fill = b4c;
cr_cell.region = -chan_wall & -top & +cr_surf;

gf_cell = openmc.Cell();
gf_cell.fill = mod;
gf_cell.region = -chan_wall & +crf_surf & -cr_surf;

cr_water_cell = openmc.Cell();
cr_water_cell.fill = water;
cr_water_cell.region = -chan_wall & +bottom & -crf_surf;

cu = openmc.Universe();
cu.add_cells([cr_cell,gf_cell,cr_water_cell]);


mod_cell = openmc.Cell();
mod_cell.fill = mod;

all_mod = openmc.Universe();
all_mod.add_cell(mod_cell);


n_cells = 3;
pitch = 25

left = openmc.YPlane(y0=-n_cells*pitch/2.,boundary_type='reflective');
right = openmc.YPlane(y0=n_cells*pitch/2.,boundary_type='reflective');
front = openmc.XPlane(x0=n_cells*pitch/2.,boundary_type='reflective');
back = openmc.XPlane(x0=-n_cells*pitch/2.,boundary_type='reflective');

lattice = openmc.RectLattice();
lattice.dimension = [3,3];
lattice.lower_left = [-n_cells*pitch/2.,-n_cells*pitch/2.];
lattice.pitch = [pitch,pitch];

lattice.universes = [
    [fu,fu,fu],
    [fu,cu,fu],
    [fu,fu,fu]
]

lattice.outer = all_mod;

core_cell = openmc.Cell();
core_cell.fill = lattice;
core_cell.region = +left & -right & -front & +back & -top & +bottom;


root = openmc.Universe();
root.add_cells([core_cell]);

geometry = openmc.Geometry();
geometry.root_universe = root;
geometry.export_to_xml();





In [10]:
colors = {};
colors[b4c]='yellow';
colors[fuel_chan_mat]='olive';
colors[mod]='gray';
colors[water]='blue';

In [11]:
p = openmc.Plot();
p.width = [-n_cells*pitch/2.,-n_cells*pitch/2.];
p.origin = [0.,0.,L_channel/2.];
p.pixels = [400,400];
p.color_by = 'material';
p.colors = colors;

openmc.plot_inline(p);

RuntimeError: Could not find material 2 specified on cell 11